In [ ]:
import torch
import sys

from ase.build import bulk
from batch_nl import NeighbourList
from vesin import ase_neighbor_list

In [ ]:
device = "cuda:0"
radius = 3.0

In [ ]:
# --- build a simple test system: diamond C 2×2×2 ---
base = bulk("C", "diamond", a=3.57)
carbon = base
print("num_atoms:", len(carbon))


In [ ]:
# --- VK neighbour list (your class, O(N^2)) ---
atoms_list = [carbon, carbon * (2,2,2)]

positions_list = [a.positions for a in atoms_list]
cell_list = [a.cell.array for a in atoms_list]

nl = NeighbourList(
    list_of_positions=positions_list,
    list_of_cells=cell_list,
    cutoff=radius,
    device=device,
)

nl.load_data()
out = nl.calculate_neighbourlist(use_torch_compile=False)
out_matscipy = nl.get_matscipy_output_from_batch_output(*out)

print ('batch_nl (str 1): pairs', len(out_matscipy[0][0]))
print ('batch_nl (str 2): pairs', len(out_matscipy[0][1]))


In [ ]:
# VESIN ASE_style neighbour list 
i_ase, j_ase, S_ase, d_ase = ase_neighbor_list("ijSd", atoms_list[0], cutoff=radius)
print("VESIN ASE nl (str 1): pairs:", len(i_ase))

i_ase, j_ase, S_ase, d_ase = ase_neighbor_list("ijSd", atoms_list[1], cutoff=radius)
print("VESIN ASE nl (str 2): pairs:", len(i_ase))